In [1]:
# ROS2 Jazzy Advanced Features Test
# Tests: Services, Actions, Parameters, QoS, Lifecycle, Transforms

import rclpy
from rclpy.node import Node
from rclpy.action import ActionServer, ActionClient
from rclpy.executors import MultiThreadedExecutor
from rclpy.callback_groups import ReentrantCallbackGroup
from rclpy.qos import QoSProfile, ReliabilityPolicy, DurabilityPolicy
from rclpy.parameter import Parameter
from rclpy.lifecycle import LifecycleNode, LifecycleState, TransitionCallbackReturn

from std_msgs.msg import String, Int32
from std_srvs.srv import SetBool
from example_interfaces.action import Fibonacci
from example_interfaces.srv import AddTwoInts
from geometry_msgs.msg import TransformStamped
from tf2_ros import TransformBroadcaster, Buffer, TransformListener

import time
import threading
from math import sin, cos

class AdvancedROS2Node(Node):
    def __init__(self):
        super().__init__('advanced_test_node')
        
        # Callback group for concurrent execution
        self.cb_group = ReentrantCallbackGroup()
        
        # 1. Custom QoS Profile
        self.setup_qos_publisher()
        
        # 2. Parameters
        self.setup_parameters()
        
        # 3. Services
        self.setup_services()
        
        # 4. Actions
        self.setup_actions()
        
        # 5. Transform broadcasting
        self.setup_transforms()
        
        # Test counters
        self.test_results = {
            'qos_publisher': False,
            'parameters': False,
            'service_server': False,
            'service_client': False,
            'action_server': False,
            'transforms': False
        }
        
        self.get_logger().info("🚀 Advanced ROS2 node initialized!")

    def setup_qos_publisher(self):
        """Test custom QoS profiles"""
        qos_profile = QoSProfile(
            reliability=ReliabilityPolicy.RELIABLE,
            durability=DurabilityPolicy.TRANSIENT_LOCAL,
            depth=10
        )
        
        self.qos_pub = self.create_publisher(
            String, 
            'qos_test_topic', 
            qos_profile,
            callback_group=self.cb_group
        )
        
        self.qos_timer = self.create_timer(
            1.0, 
            self.qos_callback,
            callback_group=self.cb_group
        )
        self.qos_count = 0

    def qos_callback(self):
        msg = String()
        msg.data = f'QoS Message {self.qos_count}'
        self.qos_pub.publish(msg)
        self.get_logger().info(f'📡 Published QoS: {msg.data}')
        self.qos_count += 1
        self.test_results['qos_publisher'] = True

    def setup_parameters(self):
        """Test parameter management"""
        # Declare parameters with defaults
        self.declare_parameter('test_string', 'default_value')
        self.declare_parameter('test_int', 42)
        self.declare_parameter('test_double', 3.14)
        self.declare_parameter('test_bool', True)
        
        # Get parameter values
        string_param = self.get_parameter('test_string').value
        int_param = self.get_parameter('test_int').value
        
        self.get_logger().info(f'📋 Parameters - String: {string_param}, Int: {int_param}')
        
        # Set new parameter values
        self.set_parameters([
            Parameter('test_string', Parameter.Type.STRING, 'updated_value'),
            Parameter('test_int', Parameter.Type.INTEGER, 100)
        ])
        
        self.test_results['parameters'] = True

    def setup_services(self):
        """Test service server and client"""
        # Service server
        self.srv_server = self.create_service(
            AddTwoInts,
            'add_two_ints',
            self.add_two_ints_callback,
            callback_group=self.cb_group
        )
        
        # Service client
        self.srv_client = self.create_client(AddTwoInts, 'add_two_ints')
        
        self.get_logger().info('🔧 Service server created')
        self.test_results['service_server'] = True

    def add_two_ints_callback(self, request, response):
        response.sum = request.a + request.b
        self.get_logger().info(f'🔧 Service call: {request.a} + {request.b} = {response.sum}')
        return response

    async def test_service_client(self):
        """Test service client call"""
        if not self.srv_client.wait_for_service(timeout_sec=2.0):
            self.get_logger().error('Service not available')
            return
        
        request = AddTwoInts.Request()
        request.a = 15
        request.b = 27
        
        future = self.srv_client.call_async(request)
        
        # Wait for response
        start_time = time.time()
        while not future.done() and (time.time() - start_time) < 5.0:
            rclpy.spin_once(self, timeout_sec=0.1)
        
        if future.done():
            response = future.result()
            self.get_logger().info(f'🔧 Service response: {response.sum}')
            self.test_results['service_client'] = True

    def setup_actions(self):
        """Test action server"""
        self.action_server = ActionServer(
            self,
            Fibonacci,
            'fibonacci',
            self.fibonacci_callback,
            callback_group=self.cb_group
        )
        
        self.get_logger().info('⚡ Action server created')
        self.test_results['action_server'] = True

    def fibonacci_callback(self, goal_handle):
        """Action server callback"""
        self.get_logger().info('⚡ Executing Fibonacci action...')
        
        feedback_msg = Fibonacci.Feedback()
        feedback_msg.partial_sequence = [0, 1]
        
        for i in range(1, min(goal_handle.request.order, 5)):  # Limit for quick test
            feedback_msg.partial_sequence.append(
                feedback_msg.partial_sequence[i] + feedback_msg.partial_sequence[i-1]
            )
            goal_handle.publish_feedback(feedback_msg)
            time.sleep(0.2)
        
        goal_handle.succeed()
        result = Fibonacci.Result()
        result.sequence = feedback_msg.partial_sequence
        return result

    def setup_transforms(self):
        """Test TF2 transforms"""
        self.tf_broadcaster = TransformBroadcaster(self)
        self.tf_timer = self.create_timer(
            0.1, 
            self.broadcast_transform,
            callback_group=self.cb_group
        )
        
        # TF2 listener
        self.tf_buffer = Buffer()
        self.tf_listener = TransformListener(self.tf_buffer, self)
        
        self.get_logger().info('🌐 Transform broadcaster/listener created')
        self.test_results['transforms'] = True

    def broadcast_transform(self):
        """Broadcast a test transform"""
        t = TransformStamped()
        t.header.stamp = self.get_clock().now().to_msg()
        t.header.frame_id = 'world'
        t.child_frame_id = 'test_frame'
        
        # Circular motion
        angle = time.time() * 2.0
        t.transform.translation.x = cos(angle) * 2.0
        t.transform.translation.y = sin(angle) * 2.0
        t.transform.translation.z = 0.0
        
        t.transform.rotation.x = 0.0
        t.transform.rotation.y = 0.0
        t.transform.rotation.z = sin(angle/2.0)
        t.transform.rotation.w = cos(angle/2.0)
        
        self.tf_broadcaster.sendTransform(t)

def test_lifecycle_node():
    """Test lifecycle management"""
    class TestLifecycleNode(LifecycleNode):
        def __init__(self):
            super().__init__('test_lifecycle_node')
            
        def on_configure(self, state: LifecycleState) -> TransitionCallbackReturn:
            self.get_logger().info('🔄 Lifecycle: Configuring...')
            return TransitionCallbackReturn.SUCCESS
            
        def on_activate(self, state: LifecycleState) -> TransitionCallbackReturn:
            self.get_logger().info('🔄 Lifecycle: Activating...')
            return TransitionCallbackReturn.SUCCESS
    
    return TestLifecycleNode()

async def test_action_client(node):
    """Test action client"""
    action_client = ActionClient(node, Fibonacci, 'fibonacci')
    
    if not action_client.wait_for_server(timeout_sec=2.0):
        node.get_logger().error('Action server not available')
        return
    
    goal_msg = Fibonacci.Goal()
    goal_msg.order = 4
    
    future = action_client.send_goal_async(goal_msg)
    
    # Wait for goal response
    start_time = time.time()
    while not future.done() and (time.time() - start_time) < 3.0:
        rclpy.spin_once(node, timeout_sec=0.1)
    
    if future.done():
        goal_handle = future.result()
        if goal_handle.accepted:
            node.get_logger().info('⚡ Action goal accepted')
            
            # Wait for result
            result_future = goal_handle.get_result_async()
            start_time = time.time()
            while not result_future.done() and (time.time() - start_time) < 3.0:
                rclpy.spin_once(node, timeout_sec=0.1)
            
            if result_future.done():
                result = result_future.result().result
                node.get_logger().info(f'⚡ Action result: {result.sequence}')

def main():
    print("🧪 Testing Advanced ROS2 Jazzy Features...\n")
    
    # Initialize ROS2
    rclpy.init()
    
    try:
        # Create main node
        node = AdvancedROS2Node()
        
        # Create lifecycle node
        lifecycle_node = test_lifecycle_node()
        
        # Multi-threaded executor for concurrent operations
        executor = MultiThreadedExecutor(num_threads=4)
        executor.add_node(node)
        executor.add_node(lifecycle_node)
        
        # Run tests for 8 seconds
        def run_tests():
            time.sleep(2)  # Let everything initialize
            
            # Test service client
            rclpy.get_global_executor().create_task(node.test_service_client())
            
            time.sleep(1)
            
            # Test action client
            rclpy.get_global_executor().create_task(test_action_client(node))
        
        # Start test thread
        test_thread = threading.Thread(target=run_tests)
        test_thread.start()
        
        # Spin for 8 seconds
        start_time = time.time()
        while time.time() - start_time < 8.0:
            executor.spin_once(timeout_sec=0.1)
        
        test_thread.join()
        
        # Print results
        print("\n" + "="*50)
        print("🏁 TEST RESULTS:")
        print("="*50)
        
        for test_name, passed in node.test_results.items():
            status = "✅ PASSED" if passed else "❌ FAILED"
            print(f"{test_name:20} : {status}")
        
        passed_tests = sum(node.test_results.values())
        total_tests = len(node.test_results)
        
        print(f"\nOverall: {passed_tests}/{total_tests} tests passed")
        
        if passed_tests == total_tests:
            print("🎉 All advanced ROS2 features working correctly!")
        else:
            print("⚠️  Some features need attention")
            
    except Exception as e:
        print(f"❌ Test failed with error: {e}")
        
    finally:
        # Cleanup
        try:
            node.destroy_node()
            lifecycle_node.destroy_node()
        except:
            pass
        rclpy.shutdown()

if __name__ == '__main__':
    main()

# Run the advanced test
main()

ModuleNotFoundError: No module named 'example_interfaces'

In [4]:
3+3

6

this is edit mode